# **Eficiencia logística basada en datos:** caso de empresa global de e-commerce

# **Descripción**

La empresa internacional de e-commerce especializada en productos electrónicos enfrenta el reto de reducir sus bloques de almacenamiento de seis (A–F) a tres (A–C) por restricciones presupuestales. Para lograrlo, busca apoyarse en un proceso de integración de datos multidimensional que le permita reorganizar su operación sin afectar la eficiencia ni la satisfacción del cliente.

El dataset disponible, con 10.999 observaciones y 12 variables, ofrece información clave sobre clientes, productos, envíos y tiempos de entrega, lo que facilita identificar patrones y diseñar estrategias para optimizar la logística. Resolver esta situación es crucial, ya que una correcta redistribución reducirá costos, mejorará los tiempos de entrega y fortalecerá la competitividad de la compañía en el mercado global de e-commerce.


---

## **Sobre la base de datos**


El [dataset](https://www.kaggle.com/datasets/prachi13/customer-analytics) cuenta con 12 variables que describen tanto características de los clientes como de los productos y la logística. Incluye un ID único por cliente, el bloque de almacén (A–E) y el modo de envío (Ship, Flight o Road). También registra el número de llamadas al servicio al cliente, la calificación del cliente (1–5), el costo del producto en dólares, las compras previas, el nivel de importancia del producto (Low, Medium, High), el género del cliente, el descuento aplicado, el peso en gramos y, finalmente, la variable objetivo Reached on time, que indica si el producto llegó a tiempo (0) o no (1).





# **Carga de librerías y datos**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
import random as rnd #genera numeros entre 0 y 1
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
nxl = '/content/drive/MyDrive/Semestre 8/Integración/retico/Train.csv'
XDB = pd.read_csv(nxl)

# **Caracterización**
Debemos realizar una caracterización estadística de las variables dentro de cada bloque actual. Es decir, analizar cómo se comportan las variables cuantitativas y cualitativas en cada bloque, para luego tener una base sólida al momento de agruparlos o redistribuirlos.

## General

In [4]:
XDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
 11  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1.0+ MB


In [6]:
XDB.describe()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
count,10999.00000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000
mean,5500.00000,4.054459,2.990545,210.196836,3.567597,13.373216,3634.016729,0.596691
std,3175.28214,1.141490,1.413603,48.063272,1.522860,16.205527,1635.377251,0.490584
min,1.00000,2.000000,1.000000,96.000000,2.000000,1.000000,1001.000000,0.000000
25%,2750.50000,3.000000,2.000000,169.000000,3.000000,4.000000,1839.500000,0.000000
50%,5500.00000,4.000000,3.000000,214.000000,3.000000,7.000000,4149.000000,1.000000
75%,8249.50000,5.000000,4.000000,251.000000,4.000000,10.000000,5050.000000,1.000000
max,10999.00000,7.000000,5.000000,310.000000,10.000000,65.000000,7846.000000,1.000000


In [7]:
XDB.columns

Index(['ID', 'Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_gms',
       'Reached.on.Time_Y.N'],
      dtype='object')

In [22]:
XDB["Warehouse_block"].unique()

array(['D', 'F', 'A', 'B', 'C'], dtype=object)

## Categóricas

In [29]:
freq_table_Mode_of_Shipment = pd.crosstab(XDB["Warehouse_block"], XDB["Mode_of_Shipment"], normalize='index') * 100
freq_table_Product_importance = pd.crosstab(XDB["Warehouse_block"], XDB["Product_importance"], normalize='index') * 100
freq_table_Gender = pd.crosstab(XDB["Warehouse_block"], XDB["Gender"], normalize='index') * 100

print(freq_table_Mode_of_Shipment)
print(freq_table_Product_importance)
print(freq_table_Gender)

Mode_of_Shipment     Flight       Road       Ship
Warehouse_block                                  
A                 16.202946  16.039280  67.757774
B                 16.148391  16.039280  67.812330
C                 16.093835  16.039280  67.866885
D                 16.194111  15.921483  67.884406
F                 16.148391  15.984724  67.866885
Product_importance      high        low     medium
Warehouse_block                                   
A                   9.001637  49.590835  41.407529
B                   7.965085  46.644845  45.390071
C                   9.165303  47.681397  43.153301
D                   9.051254  48.146129  42.802617
F                   8.265139  48.445172  43.289689
Gender                   F          M
Warehouse_block                      
A                50.627387  49.372613
B                49.536279  50.463721
C                50.245499  49.754501
D                50.872410  49.127590
F                50.600109  49.399891


## Numéricas

In [30]:
numeric_vars = ['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']

# Función para caracterizar cada variable
def describe_numeric(data, var):
    serie = data[var]
    descripcion = {
        'Variable': var,
        'Media': np.mean(serie),
        'Desviación estándar': np.std(serie, ddof=1),
        'Asimetría': skew(serie),
        'Curtosis': kurtosis(serie)
    }
    return descripcion

# Aplicar a todas las variables numéricas
resultados = pd.DataFrame([describe_numeric(XDB, var) for var in numeric_vars])

print(resultados)

              Variable        Media  Desviación estándar  Asimetría  Curtosis
0  Customer_care_calls     4.054459             1.141490   0.391872 -0.309400
1      Customer_rating     2.990545             1.413603   0.004359 -1.295611
2  Cost_of_the_Product   210.196836            48.063272  -0.157096 -0.972264
3      Prior_purchases     3.567597             1.522860   1.681668  4.003976
4     Discount_offered    13.373216            16.205527   1.798684  1.999131
5        Weight_in_gms  3634.016729          1635.377251  -0.249713 -1.447558


# **Determinar y crear clústers**

Dado que la empresa reducirá los bloques de A-F a A-C, necesitamos un proceso de integración de referencia. Para eso, vamos a agrupar (clusterizar) los datos de cada bloque actual y obtener un valor representativo (medoide) que sirva como referencia en la nueva distribución.

En el proceso de clusterización mediante, la variable Warehouse_block no se utiliza directamente para agrupar, ya que precisamente buscamos reducir sus seis categorías a tres bloques representativos. Para formar los clusters se consideran únicamente las variables numéricas del dataset —como Cost_of_the_Product, Discount_offered, Weight_in_gms, Customer_care_calls, Prior_purchases y Customer_rating—, las cuales describen cuantitativamente a los productos y clientes. Estas variables permiten calcular similitudes entre observaciones y, a partir de ello, reorganizar los bloques en tres nuevos grupos de referencia más eficientes.

In [31]:
XD = XDB[numeric_vars].values
yd = XDB['Reached.on.Time_Y.N'].values  # variable objetivo

np.random.seed(42)

# Valores mínimos y máximos
Xmin = np.min(XD, axis=0)
Xmax = np.max(XD, axis=0)
print("Los valores mínimos son:", Xmin)
print("Los valores máximos son:", Xmax)

# Inicializar clusters (3 en este caso, porque queremos A, B, C)
k = 3
XC = np.zeros((k, XD.shape[1]))

for j in range(k):
    # Inicialización aleatoria
    XC[j, ] = Xmin + (Xmax - Xmin) * rnd.random()
    # Opción alternativa: tomar algunos individuos como medoides iniciales
    XC[j, ] = XD[j, ]

# Asignación de clusters
fhat = np.zeros((len(XD), 1))

for idx in range(len(XD)):
    # Distancia euclídea inversa (similitud)
    VP = np.exp(-0.5 * (np.mean(((XC[:, ] - XD[idx, ]) / XC[:, ]) ** 2, axis=1)))
    nc = np.argmax(VP)   # cluster más cercano
    fhat[idx, ] = int(nc)
    # Actualización del medoide
    XC[nc, ] = (XC[nc, ] + XD[idx, ]) / 2

# Resumen por cluster
fhat2 = np.zeros((k, 2))
for j in range(k):
    filas = np.where(fhat[:, ] == j)[0]
    npx = len(filas)
    print("El número de observaciones en el bloque", j, "es", npx)

    # Conteo de Reached.on.Time_Y.N
    if npx > 0:
        fhat2[j, 1] = len(np.where(yd[filas] == 1)[0])  # No llegó a tiempo
        fhat2[j, 0] = len(np.where(yd[filas] == 0)[0])  # Sí llegó a tiempo
        fhat2[j, ] = fhat2[j, ] / np.sum(fhat2[j, ])    # Proporción

# Concatenar y resumen
XCT = np.column_stack((XC, fhat2))

dfXC = pd.DataFrame(XCT)
dfXC.columns = numeric_vars + ['OnTime_Prop', 'Late_Prop']
dfXC.index = ['Cluster A', 'Cluster B', 'Cluster C']

display(dfXC)

Los valores mínimos son: [   2    1   96    2    1 1001]
Los valores máximos son: [   7    5  310   10   65 7846]
El número de observaciones en el bloque 0 es 3494
El número de observaciones en el bloque 1 es 4423
El número de observaciones en el bloque 2 es 3082


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,OnTime_Prop,Late_Prop
Cluster A,4.449562,1.001030,253.270129,5.269662,1.012622,1381.870639,0.477676,0.522324
Cluster B,3.432319,3.719437,192.985053,5.262020,4.812524,1425.684728,0.239882,0.760118
Cluster C,4.617830,1.051206,243.202633,5.050843,2.675291,1670.335695,0.553537,0.446463


# **Integración de Datos**
El siguiente paso es la integración. La integración consiste en reasignar los datos de los bloques originales C, D, E y F hacia los nuevos bloques de referencia A, B y C obtenidos con K-Medoids. Para ello, se calcula el grado de pertenencia de cada observación a los tres clusters a partir de la distancia a sus medoides, y cada dato se integra en el bloque con el que tenga la mayor similitud, consolidando así la estructura de solo tres bloques.

In [32]:
# Datos de referencia (medoides iniciales: clusters A, B, C)
XC2 = np.copy(XC)

# Subconjunto de datos a integrar: bloques C, D, E, F
XD2 = XDB[XDB['Warehouse_block'].isin(['C', 'D', 'E', 'F'])][numeric_vars].values

# Proceso de integración
for k in range(len(XD2)):
    # Grado de pertenencia a cada cluster
    VP2 = np.exp(-0.5 * (np.mean(((XC - XD2[k, ]) / XC) ** 2, axis=1)))
    VP2Ing = np.exp(-0.5 * (((XC[:, 2] - XD2[k, 2]) / XC[:, 2]) ** 2))  # aquí se usa la variable "Cost_of_the_Product"

    # Condición de integración: si la pertenencia a algún cluster es suficientemente alta
    if np.max(VP2Ing) > 0.95:
        nc2 = np.argmax(VP2)  # cluster más cercano
        print("El producto", k, "pertenece al bloque", nc2)

        # Actualizamos el medoide correspondiente
        XC2[nc2, ] = (XC2[nc2, ] + XD2[k, ]) / 2

# Mostrar clusters originales (medoides iniciales)
dfXC = pd.DataFrame(XC, columns=numeric_vars, index=['Cluster A', 'Cluster B', 'Cluster C'])
print("Medoides originales (clusters de referencia):")
display(dfXC)

# Mostrar clusters modificados tras la integración
dfXC2 = pd.DataFrame(XC2, columns=numeric_vars, index=['Cluster A', 'Cluster B', 'Cluster C'])
print("Medoides tras la integración:")
display(dfXC2)

# Cambios porcentuales en las variables de cada cluster
print("Cambios porcentuales en cada variable por cluster:\n")
cambios = pd.DataFrame(np.abs((XC - XC2) / XC), columns=numeric_vars, index=['Cluster A', 'Cluster B', 'Cluster C'])
display(cambios)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
El producto 2201 pertenece al bloque 1
El producto 2202 pertenece al bloque 1
El producto 2203 pertenece al bloque 1
El producto 2204 pertenece al bloque 1
El producto 2205 pertenece al bloque 1
El producto 2206 pertenece al bloque 1
El producto 2207 pertenece al bloque 2
El producto 2208 pertenece al bloque 1
El producto 2209 pertenece al bloque 1
El producto 2210 pertenece al bloque 1
El producto 2211 pertenece al bloque 1
El producto 2212 pertenece al bloque 2
El producto 2213 pertenece al bloque 1
El producto 2214 pertenece al bloque 1
El producto 2215 pertenece al bloque 2
El producto 2216 pertenece al bloque 1
El producto 2217 pertenece al bloque 1
El producto 2218 pertenece al bloque 1
El producto 2219 pertenece al bloque 1
El producto 2220 pertenece al bloque 1
El producto 2221 pertenece al bloque 1
El producto 2222 pertenece al bloque 1
El producto 2223 pertenece al bloque 1
El producto 2224 pertenece al bloque

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
Cluster A,4.449562,1.001030,253.270129,5.269662,1.012622,1381.870639
Cluster B,3.432319,3.719437,192.985053,5.262020,4.812524,1425.684728
Cluster C,4.617830,1.051206,243.202633,5.050843,2.675291,1670.335695


Medoides tras la integración:


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
Cluster A,4.893701,1.130426,256.791361,5.654461,1.000000,1184.901461
Cluster B,3.490683,3.849612,194.302601,5.283210,4.919981,1426.868946
Cluster C,4.485097,1.067390,238.735014,4.832296,3.430697,2506.881557


Cambios porcentuales en cada variable por cluster:



,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
Cluster A,0.099816,0.129263,0.013903,0.073022,0.012464,0.142538
Cluster B,0.017004,0.034999,0.006827,0.004027,0.022329,0.000831
Cluster C,0.028744,0.015396,0.018370,0.043269,0.282364,0.500825


# **Resumen y conclusiones**
Antes de la integración, se caracterizaron los bloques originales de la empresa (A, B, C, D, F) utilizando tanto variables categóricas (Mode_of_Shipment, Product_importance, Gender) como numéricas (Customer_care_calls, Customer_rating, Cost_of_the_Product, Prior_purchases, Discount_offered, Weight_in_gms). Se calcularon proporciones para las variables categóricas y medidas estadísticas como media, desviación estándar, asimetría y curtosis para las variables numéricas, con el objetivo de entender la distribución y la heterogeneidad dentro de cada bloque. Posteriormente, se aplicó un proceso de clusterización mediante K-Medoids considerando únicamente las variables numéricas, generando tres clusters de referencia que actuarían como los nuevos bloques A, B y C.

Los clusters resultantes mostraron diferencias importantes en las características de los productos y clientes. Por ejemplo, el Cluster A se caracterizó por un mayor promedio de Customer_care_calls y Cost_of_the_Product, mientras que el Cluster B concentró productos con menor costo y mayores proporciones de retrasos en la entrega. Tras la integración de los datos de los bloques originales C, D, E y F hacia los clusters de referencia, se observaron cambios en los medoides: en particular, el Cluster C presentó un incremento significativo en Weight_in_gms y Discount_offered, reflejando la incorporación de productos más pesados y con mayores descuentos desde los bloques integrados. Los cambios porcentuales confirmaron que los bloques A y B se mantuvieron relativamente estables, mientras que el Cluster C experimentó ajustes más notables, especialmente en el peso de los productos y el descuento aplicado.

El proceso permitió reducir de seis a tres bloques de manera eficiente, manteniendo la representatividad de los datos y preservando las características principales de cada grupo. La integración mediante K-Medoids facilitó identificar los productos más representativos de cada bloque y reasignar los datos de los bloques originales menos estratégicos (C, D, E, F) a los clusters de referencia. Esto proporciona a la empresa un marco consolidado para la reorganización logística, optimización de inventarios y análisis de desempeño de los productos por bloque, asegurando que las decisiones operativas se basen en datos consistentes y representativos.